In [1]:
# 라이브러리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta

import os


%matplotlib inline

# 경고메세지 팝업 끄기
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# 데이터 불러오기

In [5]:
df=pd.read_csv('fda_train.csv')
df

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000
...,...,...,...,...,...,...,...,...,...
6556608,2136,4663855,d,2019-02-28,23:20,0,제주 제주시,기타 주점업,-4500.000000
6556609,2136,4663855,d,2019-02-28,23:24,0,제주 제주시,기타 주점업,4142.857143
6556610,2136,4663489,a,2019-02-28,23:24,0,제주 제주시,기타 주점업,4500.000000
6556611,2136,4663856,d,2019-02-28,23:27,0,제주 제주시,기타 주점업,571.428571


# 데이터 전처리

## amount 데이터 타입 변경

In [6]:
df['amount'] = df['amount'].astype(int)

## 날짜 및 지역 관련 데이터 처리

In [9]:
df["date"] = df["transacted_date"]+ " " + df["transacted_time"]

In [10]:
df["date"]  = pd.to_datetime(df["date"])

### 연, 월, 요일, 시간 , 휴일, 분기, 계절, 지역 데이터 분리

In [11]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["dayofweek"] = df["date"].dt.weekday # 0-월요일, 1-화요일
df["hour"] = df["date"].dt.hour

In [12]:
def holidays(dayofweek):
    if dayofweek in [0,1,2,3,4]:
        return "0"
    elif dayofweek in [5,6]:
        return "1"

df["holidays"] = df.dayofweek.apply(holidays)

In [13]:
# 분기
df["quarter"] = df["date"].dt.quarter

In [14]:
# 계절 
def season(month):
    season = 'winter'
    if 3 <= month <=5:
        season = 'spring'
    elif 6<= month <=8:
        season = 'summer'
    elif 9<= month <=11:
        season = 'fall'
    
    return season

df["season"] = df.month.apply(season)

In [15]:
df["sido"]= df["region"].str.split(" ").str[0]
df["city"]= df["region"].str.split(" ").str[1]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 19 columns):
 #   Column            Dtype         
---  ------            -----         
 0   store_id          int64         
 1   card_id           int64         
 2   card_company      object        
 3   transacted_date   object        
 4   transacted_time   object        
 5   installment_term  int64         
 6   region            object        
 7   type_of_business  object        
 8   amount            int32         
 9   date              datetime64[ns]
 10  year              int64         
 11  month             int64         
 12  dayofweek         int64         
 13  hour              int64         
 14  holidays          object        
 15  quarter           int64         
 16  season            object        
 17  sido              object        
 18  city              object        
dtypes: datetime64[ns](1), int32(1), int64(8), object(9)
memory usage: 925.4+ MB


In [17]:
df.head(3)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,year,month,dayofweek,hour,holidays,quarter,season,sido,city
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,2016,6,2,13,0,2,summer,NaN,NaN
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,2016,6,2,18,0,2,summer,NaN,NaN
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,2016,6,2,18,0,2,summer,NaN,NaN


## 특일 데이터 적용

In [158]:
df.iloc[0]

store_id                              0
card_id                               0
card_company                          b
transacted_date              2016-06-01
transacted_time                   13:13
installment_term                      0
region                              NaN
type_of_business                 기타 미용업
amount                             1857
date                2016-06-01 13:13:00
year                               2016
month                                 6
dayofweek                             2
hour                                 13
holidays                              0
quarter                               2
season                           summer
sido                                NaN
city                                NaN
transacted_date1    2016-06-01 00:00:00
Name: 0, dtype: object

In [135]:
do = pd.read_csv('holiday.csv')
do

,일자 및 요일,요일구분,비고
0,2016-01-01,금요일,신정
1,2016-02-08,월요일,설날
2,2016-02-09,화요일,설날
3,2016-02-10,수요일,설날
4,2016-03-01,화요일,삼일절
...,...,...,...
56,2019-09-13,금요일,추석
57,2019-10-03,목요일,개천절
58,2019-10-09,수요일,한글날
59,2019-12-25,수요일,성탄절


x=do[do["비고"]=='연말휴장일'].index
do = do.drop(x)

In [136]:
### 2019년 3월 이후 데이터 삭제 (df데이터와 일수 맞추기 ) # 위에 것 안해도 됨
do = do.truncate(after = 49)

In [159]:
do = do.truncate(before = 8)

In [139]:
do.rename(columns={'일자 및 요일':'transacted_date'}, inplace=True)

In [153]:
do.rename(columns={'요일구분':'day'}, inplace=True)

In [140]:
do["date"] = pd.to_datetime(do["transacted_date"])


In [141]:
do["dayofweek"] = do["date"].dt.weekday
do

,transacted_date,요일구분,비고,date,dayofweek
0,2016-01-01,금요일,신정,2016-01-01,4
1,2016-02-08,월요일,설날,2016-02-08,0
2,2016-02-09,화요일,설날,2016-02-09,1
3,2016-02-10,수요일,설날,2016-02-10,2
4,2016-03-01,화요일,삼일절,2016-03-01,1
5,2016-04-13,수요일,국회의원 총선거,2016-04-13,2
6,2016-05-05,목요일,어린이날,2016-05-05,3
7,2016-05-06,금요일,임시공휴일,2016-05-06,4
8,2016-06-06,월요일,현충일,2016-06-06,0
9,2016-08-15,월요일,광복절,2016-08-15,0


In [142]:
def holidays(dayofweek):
    if dayofweek in [0,1,2,3,4]:
        return "1"
    else:
        return "0"

do["holidays"] = do.dayofweek.apply(holidays)
do.head()

,transacted_date,요일구분,비고,date,dayofweek,holidays
0,2016-01-01,금요일,신정,2016-01-01,4,1
1,2016-02-08,월요일,설날,2016-02-08,0,1
2,2016-02-09,화요일,설날,2016-02-09,1,1
3,2016-02-10,수요일,설날,2016-02-10,2,1
4,2016-03-01,화요일,삼일절,2016-03-01,1,1


In [150]:
do.drop(["day","date","dayofweek"], inplace = True)
do

KeyError: "['요일구분' 'date' 'dayofweek'] not found in axis"

In [178]:
df= pd.merge(df,do, how='outer', on='transacted_date')
df

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,...,dayofweek,hour,holidays_x,quarter,season,sido,city,transacted_date1,비고,holidays_y
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,...,2,13,0,2,summer,NaN,NaN,2016-06-01,NaN,NaN
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,...,2,18,0,2,summer,NaN,NaN,2016-06-01,NaN,NaN
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,...,2,18,0,2,summer,NaN,NaN,2016-06-01,NaN,NaN
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857,2016-06-01 20:22:00,...,2,20,0,2,summer,NaN,NaN,2016-06-01,NaN,NaN
4,1,1931,b,2016-06-01,13:31,0,NaN,NaN,1071,2016-06-01 13:31:00,...,2,13,0,2,summer,NaN,NaN,2016-06-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6556608,2136,4659988,f,2017-10-04,23:32,0,제주 제주시,기타 주점업,9571,2017-10-04 23:32:00,...,2,23,0,4,fall,제주,제주시,2017-10-04,추석,1
6556609,2136,4659465,f,2017-10-04,23:41,0,제주 제주시,기타 주점업,7000,2017-10-04 23:41:00,...,2,23,0,4,fall,제주,제주시,2017-10-04,추석,1
6556610,2136,4659989,d,2017-10-04,23:47,0,제주 제주시,기타 주점업,5000,2017-10-04 23:47:00,...,2,23,0,4,fall,제주,제주시,2017-10-04,추석,1
6556611,2136,4659710,g,2017-10-04,23:53,0,제주 제주시,기타 주점업,3785,2017-10-04 23:53:00,...,2,23,0,4,fall,제주,제주시,2017-10-04,추석,1


### merge 관련 결측치 정리

In [179]:
df["비고"] = df["비고"].fillna('.')


In [180]:
df["holidays_y"] = df["holidays_y"].fillna('0')
df.sample(3)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,...,dayofweek,hour,holidays_x,quarter,season,sido,city,transacted_date1,비고,holidays_y
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,...,2,13,0,2,summer,NaN,NaN,2016-06-01,.,0
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,...,2,18,0,2,summer,NaN,NaN,2016-06-01,.,0
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,...,2,18,0,2,summer,NaN,NaN,2016-06-01,.,0


In [181]:
df = df.astype({'holidays_x':'int', 'holidays_y':'int'})

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6556613 entries, 0 to 6556612
Data columns (total 22 columns):
 #   Column            Dtype         
---  ------            -----         
 0   store_id          int64         
 1   card_id           int64         
 2   card_company      object        
 3   transacted_date   object        
 4   transacted_time   object        
 5   installment_term  int64         
 6   region            object        
 7   type_of_business  object        
 8   amount            int32         
 9   date              datetime64[ns]
 10  year              int64         
 11  month             int64         
 12  dayofweek         int64         
 13  hour              int64         
 14  holidays_x        int32         
 15  quarter           int64         
 16  season            object        
 17  sido              object        
 18  city              object        
 19  transacted_date1  datetime64[ns]
 20  비고                object        
 21  holidays

In [171]:
def holidays(holidays_x, holidays_y):
    if df7['holidays_x'] + df7['holidays_y']  >=1 :
        return "1"
    else df7['holidays_x'] + df7['holidays_y']< 1 :
        return "0"

do["holidays"] = do.dayofweek.apply(holidays)
do.head()

SyntaxError: invalid syntax (<ipython-input-171-a0e4ecb43316>, line 4)

In [ ]:
df7['holidays_x'] + df7['holidays_y']  >=1

## 결측치 제거

### region 과 type_of_business의 결측치
* region과 type_of_business 중에 1개라도 결측치면 제거
* 전체 데이터 중 약 40% 남음

In [ ]:
# missing data
total = df.isnull().sum().sort_values(ascending =False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_data.head(10)

In [ ]:
df[df["region"].isnull() | df["type_of_business"].isnull()]

In [ ]:
df_0=df[df["region"].isnull() | df["type_of_business"].isnull()].index
df = df.drop(df_0)

In [ ]:
df.isnull().sum()

In [ ]:
df_2.info()

In [ ]:
# missing data check
total = df_2.isnull().sum().sort_values(ascending =False)
percent = (df_2.isnull().sum()/df_2.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_data.head(10)

In [ ]:
df.shape

In [ ]:
2318410 - 22

### (N/Y)amount 값 중  환불데이터가 아닌 음수인 값 제거

In [ ]:
test = df.groupby(['store_id','card_id'],)['amount'].sum().to_frame()
test

In [ ]:
test0 = test[test['amount']<0]
test0.shape

In [ ]:
df[df.index == 322529]

In [ ]:
 test[test['amount']<0]

In [ ]:
df[(df["store_id"]==142) & (df["card_id"]==322529)] 

In [ ]:
df[(df["store_id"]==496) & (df["card_id"]==306712)] 

In [ ]:
df[(df["store_id"]==616) & (df["card_id"]==1197724)] 

In [ ]:
df[(df["store_id"]==770) & (df["card_id"]==1617416)] 

In [ ]:
df[(df["store_id"]==838) & (df["card_id"]==1785941)] 

In [ ]:
df[(df["store_id"]==838) & (df["card_id"]==1785956)] 

In [ ]:
df[(df["store_id"]==892) & (df["card_id"]==1896816)] 

In [ ]:
df[(df["store_id"]==977) & (df["card_id"]==2100492)] 

In [ ]:
df[(df["store_id"]==1100) & (df["card_id"]==2374816)] 

In [ ]:
df[(df["store_id"]==1354) & (df["card_id"]==2920340)] 

In [ ]:
df[(df["store_id"]==1366) & (df["card_id"]==2940184)] 

In [ ]:
df[(df["store_id"]==1538) & (df["card_id"]==3330482)]

In [ ]:
df[(df["store_id"]==1637) & (df["card_id"]==1537277)] 

In [ ]:
df[(df["store_id"]==1637) & (df["card_id"]==3549990)] 

In [ ]:
df[(df["store_id"]==1670) & (df["card_id"]==3593810)] 

In [ ]:
df[(df["store_id"]==1917) & (df["card_id"]==4138817)] 

In [ ]:
df[(df["store_id"]==1957) & (df["card_id"]==4232623)] 

In [ ]:
df[(df["store_id"]==2128) & (df["card_id"]==4644633)] 

In [ ]:
df.index

In [ ]:
df[df.index == 322529]

##  type_of_business 전처리

### 중분류 컬럼 만들기

In [ ]:
df['type_of_business'].unique()

In [ ]:
biz = pd.read_csv('businesstype.csv')
biz

In [ ]:
biz.fillna(method='ffill')

In [ ]:
for i in range(len(df)):
    for j in range(len(biz)):
        if df['type_of_business'][i] == biz['Unnamed: 9'][j] :
            df['type_of_business'][i] = biz['Unnamed: 1'][j]

* 데이터 분리 완료 (다음에 할 때는 비슷한 요소끼리 모으기)

In [ ]:
df.to_csv('funda_data.csv', index= False)

In [ ]:
df=pd.read_csv('funda_data.csv')
df